## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,58.35,71,38,3.87,scattered clouds
1,1,Rikitea,PF,-23.1203,-134.9692,69.60,59,66,9.64,broken clouds
2,2,Presidencia Roque Saenz Pena,AR,-26.7852,-60.4388,78.89,32,54,13.06,broken clouds
3,3,Kaitangata,NZ,-46.2817,169.8464,56.82,68,50,6.20,scattered clouds
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,73.36,94,75,8.05,broken clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?80


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Presidencia Roque Saenz Pena,AR,-26.7852,-60.4388,78.89,32,54,13.06,broken clouds
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,73.36,94,75,8.05,broken clouds
8,8,Kalmunai,LK,7.4167,81.8167,78.87,80,73,4.83,broken clouds
20,20,Vaini,TO,-21.2000,-175.2000,74.35,85,85,11.81,light rain
28,28,Puerto Ayora,EC,-0.7393,-90.3518,75.16,86,81,5.01,broken clouds


In [12]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))

preferred_cities_df.count()

212


City_ID                212
City                   212
Country                210
Lat                    212
Lng                    212
Max Temp               212
Humidity               212
Cloudiness             212
Wind Speed             212
Current Description    212
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis= "index", how= "any")
clean_df.count()

City_ID                210
City                   210
Country                210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Presidencia Roque Saenz Pena,AR,78.89,broken clouds,-26.7852,-60.4388,
4,Arraial Do Cabo,BR,73.36,broken clouds,-22.9661,-42.0278,
8,Kalmunai,LK,78.87,broken clouds,7.4167,81.8167,
20,Vaini,TO,74.35,light rain,-21.2000,-175.2000,
28,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
34,Manta,EC,76.78,few clouds,-0.9500,-80.7333,
37,Ribeira Grande,PT,70.70,broken clouds,38.5167,-28.7000,
41,Gat,IL,79.39,clear sky,31.6100,34.7642,
42,Yellowknife,CA,70.32,broken clouds,62.4560,-114.3525,
47,Banda Aceh,ID,78.80,overcast clouds,5.5577,95.3222,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add lat. and long. to location key for the params dictionary
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis= "index", how= "any")
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Presidencia Roque Saenz Pena,AR,78.89,broken clouds,-26.7852,-60.4388,Hotel Gualok
4,Arraial Do Cabo,BR,73.36,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
8,Kalmunai,LK,78.87,broken clouds,7.4167,81.8167,VS Villa
20,Vaini,TO,74.35,light rain,-21.2000,-175.2000,Keleti Beach Resort
28,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
...,...,...,...,...,...,...,...
682,Bantigue,PH,77.95,light rain,10.9899,124.6225,GV Hotel Ormoc
684,Mankachar,IN,78.30,scattered clouds,25.5333,89.8667,Sanowar hussain
686,Gigmoto,PH,79.79,light rain,13.7796,124.3900,Cj's Travellers Inn
687,Alta Floresta,BR,76.51,light rain,-9.8756,-56.0861,Floresta Amazonica Hotel


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))